In [ ]:
import os
os.path.join('/home/nathanieljones/PycharmProjects/FORCE_Geolocation_Docs/')
import altair as alt
from data_load_functions import *
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster
import numpy as np

In [ ]:
map_dict = create_npd_shapefile_dict()
structures_map = map_dict['structures']
print(structures_map.crs)
filter_list = []
for idx, row in structures_map.iterrows():
    if row['geometry'] is None:
        filter_list.append(False)
    elif str(type(row['geometry'])) == '<class \'shapely.geometry.multipolygon.MultiPolygon\'>':
        filter_list.append(False)
    else:
        filter_list.append(True)
structures_map = structures_map[filter_list]
structures_map = structures_map.to_crs(epsg=4326)
structures_map.to_file("structures.geojson", driver='GeoJSON')
# map_dict['wells'].to_file("wells.geojson", driver='GeoJSON')
m = folium.Map(location=[60.5236, 5], zoom_start=6)
# folium.GeoJson(
#     'structures.geojson',
#     name='structures'
# ).add_to(m)
print('added structure polys')

wells_df = map_dict['wells']
wells_df = wells_df.to_crs(epsg=4326)
x = pd.DataFrame([wells_df.geometry.x, wells_df.geometry.y])
# x.concat(wells_df.geometry.y.values, axis=1)
print(x.shape)
# y = wells_df.geometry.y.values
# temp = np.concatenate(x,y, axis=0)
# print(temp)
mc = MarkerCluster(locations=x.T.values,
                  name='wells').add_to(m)
# wells_df.to_file("wells.geojson", driver='GeoJSON')
for idx, row in wells_df.iterrows():
    mc.add_child(folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row['wlbWell']))
# folium.GeoJson(
#     'wells.geojson',
#     name='wells',
#     zoom_start=4
# ).add_to(m)
# folium.LayerControl().add_to(m)
print('added wells')
m.add_child(mc)
print('done')
m